In [1]:
#
# this magic uses the browser backend
#
%matplotlib

Using matplotlib backend: Qt5Agg


In [2]:
#
#libraries
#
import pandas as pd
import umap
#
# umap.plot used only for the 2D network graph
#
import umap.plot
#
import numpy as np
#
# linear algebra libraries used for the ellipsoid fit
#
from numpy.linalg import eig, inv
#
import matplotlib as mpl
import matplotlib.pyplot as plt
import os
import sys
import re
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
#
# This import registers the 3D projection
#
from mpl_toolkits.mplot3d import Axes3D
#

In [3]:
#
# fit an ellipsoid to the 3D embedding
#
# from http://www.juddzone.com/ALGORITHMS/least_squares_3D_ellipsoid.html
#
# least squares fit to a 3D-ellipsoid
#
#  Ax^2 + By^2 + Cz^2 +  Dxy +  Exz +  Fyz +  Gx +  Hy +  Iz  = 1
#
# Note that sometimes it is expressed as a solution to
#
#  Ax^2 + By^2 + Cz^2 + 2Dxy + 2Exz + 2Fyz + 2Gx + 2Hy + 2Iz  = 1
#
# where the last six terms have a factor of 2 in them
# This is in anticipation of forming a matrix with the polynomial coefficients.
# Those terms with factors of 2 are all off diagonal elements.  These contribute
# two terms when multiplied out (symmetric) so would need to be divided by two

def ls_ellipsoid(xx,yy,zz):

# change xx from vector of length N to Nx1 matrix so we can use hstack

   x = xx[:,np.newaxis]
   y = yy[:,np.newaxis]
   z = zz[:,np.newaxis]

#  Ax^2 + By^2 + Cz^2 +  Dxy +  Exz +  Fyz +  Gx +  Hy +  Iz = 1

   J = np.hstack((x*x,y*y,z*z,x*y,x*z,y*z, x, y, z))
   K = np.ones_like(x) #column of ones

#np.hstack performs a loop over all samples and creates
#a row in J for each x,y,z sample:

# J[ix,0] = x[ix]*x[ix]
# J[ix,1] = y[ix]*y[ix]
# etc.

   JT=J.transpose()
   JTJ = np.dot(JT,J)
   InvJTJ=np.linalg.inv(JTJ);
   ABC= np.dot(InvJTJ, np.dot(JT,K))

# Rearrange, move the 1 to the other side
#  Ax^2 + By^2 + Cz^2 +  Dxy +  Exz +  Fyz +  Gx +  Hy +  Iz - 1 = 0
#    or
#  Ax^2 + By^2 + Cz^2 +  Dxy +  Exz +  Fyz +  Gx +  Hy +  Iz + J = 0
#  where J = -1

   eansa=np.append(ABC,-1)

   return (eansa)
#

In [4]:
#
# get prediced enclosing ellipsoid parameters
#
def get_preds(data, params):
#
#  Ax^2 + By^2 + Cz^2 +  Dxy +  Exz +  Fyz +  Gx +  Hy +  Iz - 1 = 0
#
    A = params[0]
    B = params[1]
    C = params[2]
    D = params[3]
    E = params[4]
    F = params[5]
    G = params[6]
    H = params[7]
    I = params[8]
    J = params[9]
#
    pred = []
    for i in range(data.shape[0]):
        pred.append(A * data.iloc[i, 0]**2 +
                    B * data.iloc[i, 1]**2 +
                    C * data.iloc[i, 2]**2 +
                    D * data.iloc[i, 0] * data.iloc[i, 1] +
                    E * data.iloc[i, 0] * data.iloc[i, 2] +
                    F * data.iloc[i, 1] * data.iloc[i, 2] +
                    G * data.iloc[i, 0] +
                    H * data.iloc[i, 1] +
                    I * data.iloc[i, 2] +
                    J)
    return pred
#

In [5]:
#
# center the embedding and fit the ellipsoid
#
def centered_ellipsoid(embedding):
#
# build a dataframe to do least squares fit of 3D ellipsoid
#
    fit_ellipsoid = embedding.copy()
#
# for simplicity translate to the origin
#
    fit_ellipsoid['X'] = fit_ellipsoid['X'] - fit_ellipsoid['X'].mean()
    fit_ellipsoid['Y'] = fit_ellipsoid['Y'] - fit_ellipsoid['Y'].mean()
    fit_ellipsoid['Z'] = fit_ellipsoid['Z'] - fit_ellipsoid['Z'].mean()
#
# fit this--the surface will be somewhere inside the cloud
#
    ellipsoid_params = ls_ellipsoid(np.array(fit_ellipsoid['X']),
                                   np.array(fit_ellipsoid['Y']),
                                   np.array(fit_ellipsoid['Z']))
#
    return ellipsoid_params, fit_ellipsoid
#

In [6]:
#
# for mnist demo
#
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(60000, 784)

In [7]:
#
# construct the UMAP embedding
#
reducer2D = umap.UMAP(random_state = 42,
                      n_neighbors = 30,
                      min_dist = 0.0,
                      n_components = 2)
mapper2D = reducer2D.fit(x_train)
mnist_embedding = mapper2D.transform(x_train)

In [8]:
#
# visualize results
#
plot_data = pd.concat([pd.DataFrame(mnist_embedding), 
                       pd.Series(y_train)], axis = 1)
plot_data.columns = ['X', 'Y', 'class']
centers = plot_data.groupby('class').mean()
#
plt.style.use('dark_background')
fig, ax = plt.subplots()
ax.scatter(plot_data.iloc[:, 0],
           plot_data.iloc[:, 1],
           c = y_train,
           s = 0.1,
           cmap = 'jet')
for my_class in plot_data['class'].unique():
    ax.annotate(str(my_class),
               (centers.X[my_class], centers.Y[my_class]),
               size = 20,
               color = 'black',
               ha = 'center',
               va = 'center')
plt.show()

In [9]:
#
# get data
#
raw_data = pd.read_csv('UMAP_demo_data.csv')
raw_data_cols = raw_data.columns
#
# scale
# 
raw_data = pd.DataFrame(StandardScaler().fit_transform(raw_data))
raw_data.columns = raw_data_cols
#

In [10]:
#
# UMAP
#
random_seed = 42
my_data = raw_data.copy()
#
# store the original index
#
my_index = my_data.index
#
# define the dimensionality reduction
#
reducer3D = umap.UMAP(n_neighbors = 50,
                        min_dist = 0.5,
                        n_components = 3,
                        metric = 'cosine',
                        random_state = random_seed,
                        set_op_mix_ratio = 0.01)
reducer2D = umap.UMAP(n_neighbors = 50,
                        min_dist = 0.5,
                        n_components = 2,
                        metric = 'cosine',
                        random_state = random_seed,
                        set_op_mix_ratio = 0.01)
#
# compute the embedding
#
mapper2D = reducer2D.fit(my_data)
mapper3D = reducer3D.fit(my_data)
embedding = mapper3D.transform(my_data)
#
# visualize the connectivity in the embedded space
#
umap.plot.connectivity(mapper2D, show_points = True)
#
# convert to Pandas DataFrame and label
#
embedding = pd.DataFrame(embedding)
embedding.columns = ['X', 'Y', 'Z']
#
# get an ellipsoid that fits the embedded values in 3D
#
best_ellipsoid, embedding = centered_ellipsoid(embedding)
#
# get the predicted values for the embedded data for
# the distance from the origin to the points
#
# the predicted values provide the threshold to define outliers
#
embedding['pred'] = get_preds(embedding, best_ellipsoid)
embedding['rows'] = list(my_index)
#

C:\Users\bbate\Miniconda3\envs\keras-gpu-5\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1
C:\Users\bbate\Miniconda3\envs\keras-gpu-5\lib\site-packages\matplotlib\colors.py:527: RuntimeWarning: invalid value encountered in less
  xa[xa < 0] = -1


In [11]:
#
# color mapping
#
# choose a variable from the data usd in UMAP
#
C_var = 'f_85'
C_var_scales = pd.Series({'scale' : 0.000693,
                          'mean' : 0.004019})
#
my_data = raw_data.copy()
my_data.sort_values(C_var, inplace = True, ascending = False)
#
# preserve index
#
my_index = my_data.index
my_data.reset_index(drop = True, inplace = True)
C_range = list(np.arange(my_data[C_var].min(),
                         my_data[C_var].max(),
                         (my_data[C_var].max() - my_data[C_var].min()) / 16))
#
# map the colors so the red ("hot") end of the scale
# maps to the largest values
#
my_colors = plt.cm.jet(np.linspace(0, 1, len(C_range)))
cbar_values = pd.Series(C_range) * C_var_scales.loc['scale']
cbar_values = cbar_values + C_var_scales.loc['mean']
#
C_range.reverse()
colors = []
color = len(C_range) - 1
count = 0
counts = []
for temp in C_range:
    print(temp, count)
    counts.append(count)
    while (my_data[C_var][count] >= temp):
        colors.append(color)
        count += 1
        if count >= my_data.shape[0]:
            break
    color -= 1
    if count == my_data.shape[0]:
        break
#
# counts stores how many were assigned to the bins
#
counts = counts[1: ] + [my_data.shape[0]]
counts = counts[::-1]
my_data.set_index(my_index, drop = True, inplace = True)
#
embedding['color'] = pd.Series(colors)
#

1.2995839037939043 0
1.0564970545550811 4855
0.813410205316258 4983
0.5703233560774348 5343
0.3272365068386116 9243
0.08414965759978843 11373
-0.15893719163903475 11563
-0.4020240408778579 11710
-0.6451108901166811 15073
-0.8881977393555043 17750
-1.1312845885943275 17822
-1.3743714378331506 19816
-1.6174582870719738 23972
-1.860545136310797 24847
-2.10363198554962 24901
-2.3467188347884433 24919


In [12]:
#
# visualize
#
visualize = True
if visualize:
#
# legend scale is a multiplier to put the legend labels in real units
# legend exponent can be used to re-scale features with powers otehr than 1
# legend digits is rounding of the displayed values
# legend title should communicate the variable name and units
#
    legend_scale = 1
    legend_exponent = -1
    legend_digits = 0
    legend_title = 'T1 (C)'
#
# if the color variable is inverse, then we have to
# reverse the color scale when we rescale the values
#
    if legend_exponent < 0:
        cbar_values = cbar_values.sort_values(ascending = False).reset_index(drop = True)
        embedding['color'] = 15 - embedding['color']
#
# set a threshold for excess radius
# note that roughly the fitted ellipsoid
# should be about halfway inside the data so
# a reasonable "true" surface would be about 0.5
#
    outlier_margin = 0.5
#
# inner core margin is an exlusion region in the center
# of the ellipsoid to make the plot easier to see
#
    inner_core_margin = -0.25
#
# choose to plot the interior points or not
# sample the core points to speed up plotting
#
    plot_core = True
    fraction = 0.5
    sample = np.random.choice(embedding.shape[0],
                              np.int(fraction * embedding.shape[0]),
                              replace = False)
#
# tilt is phi (tilt of the vertical axis)
# angle is theta (rotation about the vertical axis)
#
    angle = 0
    tilt = 15
#
    plt.clf()
    plt.close()
    plt.style.use('dark_background')
    fig = plt.figure()
    ax = fig.gca(projection = '3d')
    ax.set_axis_off()
#
# we plot the points inside the ellipsoid only if we choose to
# in addition we downsample to save time, and we leave
# out the ones in the middle which are low outliers
#
    if plot_core:
        cs = [i for i in range(len(sample))
             if ((embedding.loc[i, 'pred'] < outlier_margin) &
                 (embedding.loc[i, 'pred'] > inner_core_margin))]
        p = ax.scatter(embedding.loc[sample, 'X'],
                           embedding.loc[sample, 'Y'],
                           embedding.loc[sample, 'Z'],
                           c = my_colors[embedding.loc[sample, 'color']],
                           alpha = 0.25,
                           s = 7,
                           antialiased = False)
#
# plot the outliers
#
    outliers = [i for i in range(embedding.shape[0])
                if (embedding.loc[i, 'pred'] >= outlier_margin)]
    ax.scatter(embedding.loc[outliers, 'X'],
               embedding.loc[outliers, 'Y'],
               embedding.loc[outliers, 'Z'],
               c = 'magenta',
               alpha = 1,
               s = 11,
               antialiased = False)
#
# optionally plot the inner core points
#
    plot_dark_core = True
    dark_core = [i for i in range(embedding.shape[0])
                 if ((embedding.loc[i, 'pred'] >= outlier_margin) |
                     (embedding.loc[i, 'pred'] <= inner_core_margin))]
    if plot_dark_core:
        ax.scatter(embedding.loc[dark_core, 'X'],
                   embedding.loc[dark_core, 'Y'],
                   embedding.loc[dark_core, 'Z'],
                   c = 'lightgrey',
                   alpha = 0.05,
                   s = 1,
                   antialiased = False)
#
# set rotation of plot
#
    ax.view_init(tilt, angle)
    ax.set_xlabel(embedding.columns[0], fontsize = 12)
    ax.set_ylabel(embedding.columns[1], fontsize = 12)
    ax.set_zlabel(embedding.columns[2], fontsize = 12)
    ax.text2D(0.1, 0.9,
              'theta: ' + str(angle) +
              '\nphi: ' + str(tilt) +
              '\nthreshold: ' + str(outlier_margin),
              transform = ax.transAxes,
              fontsize = 14)
#
# construct a color key
#
# get the extreme corner of the space
# we use a dummy points there to generate the 
# legend entries
#
    x_max = embedding['X'].max()
    y_max = embedding['Y'].max()
    z_max = embedding['Z'].max()
    for color in range(len(cbar_values)):
        if counts[color] > 0:
            ax.scatter(x_max, y_max, z_max,
                       c = my_colors[color][0:3].reshape(1, -1),
                       label = (str(round(cbar_values[color]**legend_exponent / legend_scale,
                                          legend_digits))),
                       s = 13)
    ax.scatter(x_max, y_max, z_max,
               c = 'magenta',
               label = 'outliers',
               s = 14)
#
# draw a point to over-write the faux points we used
# to create the legend entry
#
    ax.scatter(x_max, y_max, z_max, c = "black", s = 15)
    ax.legend(loc = 'center right',
          title = legend_title)
    plt.show()